In [1]:
import requests
import json
from pprint import pp
import pandas as pd
import pyspark

In [2]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd

In [3]:
url='https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
resposne=requests.get(url)
print(resposne)
r=resposne.json()

<Response [200]>


In [14]:
#df_loaneligible=pd.DataFrame(r)
df = pd.json_normalize(r)
pp(df)

    Application_ID  Gender Married Dependents     Education Self_Employed  \
0         LP001002    Male      No          0      Graduate            No   
1         LP001003    Male     Yes          1      Graduate            No   
2         LP001005    Male     Yes          0      Graduate           Yes   
3         LP001006    Male     Yes          0  Not Graduate            No   
4         LP001008    Male      No          0      Graduate            No   
..             ...     ...     ...        ...           ...           ...   
506       LP002978  Female      No          0      Graduate            No   
507       LP002979    Male     Yes         3+      Graduate            No   
508       LP002983    Male     Yes          1      Graduate            No   
509       LP002984    Male     Yes          2      Graduate            No   
510       LP002990  Female      No          0      Graduate           Yes   

     Credit_History Property_Area  Income Application_Status  
0           

In [10]:
spark = SparkSession.builder.appName("loan").getOrCreate()

In [11]:
sploanDF=spark.createDataFrame(df) 
sploanDF.printSchema()
sploanDF.show()


c:\Python310\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Python310\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- Application_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Application_Status: string (nullable = true)

+--------------+------+-------+----------+------------+-------------+--------------+-------------+------+------------------+
|Application_ID|Gender|Married|Dependents|   Education|Self_Employed|Credit_History|Property_Area|Income|Application_Status|
+--------------+------+-------+----------+------------+-------------+--------------+-------------+------+------------------+
|      LP001002|  Male|     No|         0|    Graduate|           No|             1|        Urban|medium|                 Y|
|      LP001003|  Male|    Yes|         1|    Graduate|           

In [ ]:
jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"
table_name = 'CDW-SAPP_loan_application'
properties = {'user': 'root', 'password': 'password'}

sploanDF.write.jdbc(url=jdbc_url, table=table_name, mode='append', properties=properties)

In [15]:
sploanDF.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_loan_application") \
  .option("user", "root") \
  .option("password", "password") \
  .save()